<a href="https://colab.research.google.com/github/ibarajas248/scraping-supersociedades/blob/main/web_Scraping_supersociedades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium webdriver-manager pandas openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 40.3 MB/s eta 0:00:00


In [3]:
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt install -y ./google-chrome-stable_current_amd64.deb

--2025-07-03 14:01:19--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 172.217.164.14, 2607:f8b0:4025:815::200e
Connecting to dl.google.com (dl.google.com)|172.217.164.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118175608 (113M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>] 112.70M  25.7MB/s    in 5.3s    

2025-07-03 14:01:25 (21.4 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [118175608/118175608]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'google-chrome-stable' instead of './google-chrome-stable_current_amd64.deb'
The following additional packages will be installed:
  libvulkan1 mesa-vulkan-drivers
The following NEW packages will be installed:
  google-chrome-stable libvulkan1 mesa-vulkan-drivers
0 upgraded, 3 newly in

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import time
import pandas as pd
import random

# Configuración de opciones para usar Chrome sin interfaz gráfica
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # Opcional: ejecuta sin abrir la ventana del navegador
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36")

# Inicializar el WebDriver usando el driver configurado
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# URL de la página
url = "https://siis.ia.supersociedades.gov.co/#/dashboard"
driver.get(url)

# Esperar que el contenido dinámico se cargue
time.sleep(random.uniform(4, 7))

# Hacer clic en el botón "BUSCAR"
try:
    buscar_btn = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.CLASS_NAME, "btn-search-icon"))
    )
    buscar_btn.click()
    print("Botón 'BUSCAR' presionado.")
    time.sleep(random.uniform(3, 6))
except Exception as e:
    print(f"Error al presionar el botón 'BUSCAR': {e}")

# Inicializar lista para almacenar los datos
data = []

# Navegar por páginas
page_num = 0
max_pages = 100

try:
    while page_num < max_pages:
        div_content = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CLASS_NAME, "app-main__central"))
        )

        rows = div_content.find_elements(By.CLASS_NAME, "row")
        print(f"Filas encontradas en la página {page_num + 1}: {len(rows)}")

        for i, row in enumerate(rows):
            print(f"\nContenido del div 'row' {i + 1}:")
            try:
                item_detail_divs = row.find_elements(By.CLASS_NAME, "item-detail.subtitles")
                empresa = "No encontrado"

                if item_detail_divs:
                    for item_detail in item_detail_divs:
                        try:
                            entry_point_div = item_detail.find_element(By.CLASS_NAME, "label-default.info-entrypoint")
                            if entry_point_div:
                                entry_point_div_text = entry_point_div.text.strip()
                                item_detail_text = item_detail.text.replace(entry_point_div_text, '').strip()
                            else:
                                item_detail_text = item_detail.text.strip()
                        except:
                            item_detail_text = item_detail.text.strip()

                        empresa = item_detail_text
                        print("Empresa:", empresa)

                activos = ingresos = utilidad_neta = "No encontrado"

                economy_info_divs = row.find_elements(By.CLASS_NAME, "col-md-12.d-flex.economy-info")
                for j, economy_info in enumerate(economy_info_divs):
                    widgetnumbers = economy_info.find_elements(By.CLASS_NAME, "widget-numbers")
                    for idx, widget in enumerate(widgetnumbers):
                        label = ["Activos", "Ingresos", "Utilidad Neta"][idx] if idx < 3 else f"Valor {idx + 1}"
                        outer_html = widget.get_attribute('outerHTML')
                        match = re.search(r'(\d[\d,.]*)', outer_html)
                        value = match.group(1) if match else "No encontrado"

                        if idx == 0:
                            activos = value
                        elif idx == 1:
                            ingresos = value
                        elif idx == 2:
                            utilidad_neta = value

                        print(f"{label}: {value}")

                data.append({
                    'Empresa': empresa,
                    'Activos': activos,
                    'Ingresos': ingresos,
                    'Utilidad Neta': utilidad_neta
                })

                time.sleep(random.uniform(0.5, 1.5))  # Espera entre cada fila

            except Exception as e:
                print(f"Error al procesar el contenido del 'row' {i + 1}: {e}")

        # Intentar pasar a la siguiente página
        try:
            next_page_button = WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable((By.XPATH, "//*[@aria-label='Go to next page']"))
            )

            if "Mui-disabled" in next_page_button.get_attribute("class"):
                print("Ya estamos en la última página.")
                break
            else:
                next_page_button.click()
                print("Botón 'Siguiente página' presionado.")
                page_num += 1
                time.sleep(random.uniform(4, 7))
        except Exception as e:
            print(f"Error al presionar 'Siguiente página': {e}")
            break

except Exception as e:
    print(f"Error al obtener los divs 'row': {e}")

# Cerrar navegador
driver.quit()

# Guardar datos en Excel
df = pd.DataFrame(data)
df.to_excel("resultado.xlsx", index=False)
print("Datos guardados en 'resultado.xlsx'.")


Botón 'BUSCAR' presionado.
Filas encontradas en la página 1: 21

Contenido del div 'row' 1:
Empresa: Ingenieros servicios constructivos sas
Activos: 13.283.672
Ingresos: 2.495.661
Utilidad Neta: 24.429

Contenido del div 'row' 2:
Activos: 13.283.672
Ingresos: 2.495.661
Utilidad Neta: 24.429

Contenido del div 'row' 3:
Empresa: Construcciones industriales y servicios super sas
Activos: 21.091.663
Ingresos: 11.615.843
Utilidad Neta: 166.730

Contenido del div 'row' 4:
Activos: 21.091.663
Ingresos: 11.615.843
Utilidad Neta: 166.730

Contenido del div 'row' 5:
Empresa: Cereria el sagrado corazon ltd
Activos: 5.741.062
Ingresos: 12.200.295
Utilidad Neta: 939.640

Contenido del div 'row' 6:
Activos: 5.741.062
Ingresos: 12.200.295
Utilidad Neta: 939.640

Contenido del div 'row' 7:
Empresa: Inversiones schlegel donado sas
Activos: 7.553.038
Ingresos: 240.000
Utilidad Neta: 46.798

Contenido del div 'row' 8:
Activos: 7.553.038
Ingresos: 240.000
Utilidad Neta: 46.798

Contenido del div 'row' 9:
